# Linear Prediction: A Tutorial Review
### John Makhoul


I think this demonstration will go through the steps of LPC analysis \
https://www.mathworks.com/help/signal/ug/formant-estimation-with-lpc-coefficients.html

Here is an overview of the steps we will follow:
1. Window the signal
2. Apply pre-emphasis
3. Compute coefficients (autocorrelation)
4. Compute frequency response


In [9]:
using DSP
using WAV

In [10]:
# I will set all parameters in this cell
window_size = 400
overlap = 100
order = 10

100

In [11]:
# Load the file
signal, fs, b, opt = wavread("demo_data/aa_audio/audio_lpc_aa_fo100.wav")

([0.0; -3.051850947599719e-5; … ; 0.00012207403790398877; 0.0;;], 44100.0f0, 0x0010, WAVChunk[WAVChunk(Symbol("fmt "), UInt8[0x10, 0x00, 0x00, 0x00, 0x01, 0x00, 0x01, 0x00, 0x44, 0xac, 0x00, 0x00, 0x88, 0x58, 0x01, 0x00, 0x02, 0x00, 0x10, 0x00])])

In [12]:
# Here is the same function for Hanning windows that I used in the spectrogram code
function hanning(size)
    # where size refers to the window length
    return 0.5 * (1 .- cos.(2π * (0:size-1) ./ (size-1)))
end

hanning (generic function with 1 method)

In [13]:
# Window the signal
num_windows = div(length(signal) - window_size, window_size - overlap) + 1
windows = []

for i in 1:num_windows
    start_idx = (i - 1) * (window_size - overlap) + 1
    end_idx = start_idx + window_size - 1
    window = signal[start_idx:end_idx]
    windowed_signal = window .* hanning(window_size)
    push!(windows, windowed_signal)
end

In [ ]:
# Fuction for performing autocorrelation
# Autocorrelation is basically a mathematical way of looking for repeating patterns in the signal
# The function returns an array of autocorrelation values of length = order
function autocorrelation(signal, p_order)
    N = length(signal)
    r = zeros(Float64, p_order + 1)
    for k in 0:p_order # for every lag in order
        r[k + 1] = sum(signal[1:N - k] .* signal[1 + k:N]) # sum the product of two different sections of the signal
    end
    return r
end

In [ ]:
# Compute coefficients


In [15]:
# Test
for window in windows
    println(size(window))
end

(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)
(400,)


In [2]:
using Statistics

x1 = [6.525, 6.485, 6.543, 6.713, 6.671, 6.800, 6.824, 6.650, 6.693, 6.450]
x2 = [0.762, 0.760, 0.748, 0.756, 0.754, 0.762, 0.761, 0.747, 0.759, 0.758]
x3 = [0.581, 0.577, 0.559, 0.572, 0.569, 0.581, 0.579, 0.557, 0.576, 0.575]
x4 = [0.803, 0.796, 0.775, 0.784, 0.804, 0.804, 0.808, 0.787, 0.801, 0.794]
x5 = [0.644, 0.633, 0.600, 0.615, 0.647, 0.646, 0.653, 0.619, 0.642, 0.631]

s1 = std(x1)
s2 = std(x2)
s3 = std(x3)
s4 = std(x4)
s5 = std(x5)

println(s1)
println(s2)
println(s3)
println(s4)
println(s5)

0.12961584265307471
0.005478239782184709
0.008566082989195072
0.010616548926610347
0.016931233465600406
